Naive few-shot paht planning using LLMs

In [ ]:
openai_api_key = ""
import time
import matplotlib.pyplot as plt
import numpy as np
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)

In [ ]:
BASE_MODEL = "gpt-4-turbo"  # Update to GPT-4 model

def LLM(messages, model=BASE_MODEL, stop=None, max_tokens=256, temperature=0.0):  #Defualt = 0.9
    responses = client.chat.completions.create(
        model=model, messages=messages, max_tokens=max_tokens, 
        temperature=temperature, stop=stop
    )
    text = responses.choices[0].message.content
    return text

# Test LLM with GPT-4
LLM([{"role": "user", "content": "hello world!"}])

In [ ]:
PROMPT_dis = [
    {
        "role": "user",
        "content": "I need your assistance in planning a path based on specific instructions.\n\n"
                   "Environment details:\n"
                   "- Object 1: [name: red box, shape: square, color: red, center position: [-1.5, 1.2]]\n"
                   "- Object 2: [name: yellow box, shape: square, color: yellow, center position: [1.0, -0.5]]\n"
                   "- Object 3: [name: green box, shape: square, color: green, center position: [-1, 0]]\n"
                   "- Obstacle: [name: blue area, shape: square, color: blue, square with corners at [[-0.5, 0.0], [0.5, 0.0], [-0.5, -1.0], [0.5, -1.0]]]\n\n"

                   "**The blue area is a restricted zone and must be avoided completely during the path. No point in the path can intersect or enter this area.**"
                   
                   "Instructions:\n"
                   "- Start at the yellow box, then navigate to the green box, and finally end at the red box.\n"
                   "- Avoid entering the blue area (obstacle) at any point during the path.\n"
                   "- The path should include curved motions or additional waypoints to smoothly navigate around obstacles and restricted zones.\n\n"
                   
                   "Examples for reference:\n"
                   "- Example 1: Starting from the green box and ending at the yellow box while avoiding the blue area: [1, 0.5] -> [0.7, -1.5] -> [-0.6, -1.5] -> [-1.0, 0.0]\n"
                   "- Example 2: Starting from the green box and ending at the yellow box while avoiding the blue area: [1, 0.5] -> [0.7, 0.5] -> [-0.6, 0.55] -> [-1.0, 0.0]\n\n"
                   
                   "Task breakdown:\n"

                   "2. Identify which corners of the blue area are at the lower side (downside) and which ones are at the upper side (upside), based on their y-coordinates. Assume that a larger y-coordinate indicates the upper direction."
                   "3. 3. Identify which corners of the blue area are on the left side and which ones are on the right side, based on their x-coordinates. Assume that a larger x-coordinate indicates the right direction."
                   "4. Assign the following labels to the corners of the blue area: "
                        "- Left top corner: C1"
                        "- Right top corner: C2"
                        "- Right bottom corner: C3"
                        "- Left bottom corner: C4"

                    "5. For each corner of the blue area, the path must avoid specific directions based on the corner's position. Use the following rules for path planning:"
                        "- For C1 (left top corner): The path must stay either on its left, on its top, or both on its left and top."
                        "- The path (waypoints of path) must **never** be directly in the region below and the right of C1."
                        "- However, the path can move above and right of C1, or below and left of C1."

                        "- For C2 (right top corner): The path must stay either on its right, on its top, or both on its right and top."
                        "- The path (waypoints of path) must **never** be directly in the region below and the left of C2."
                        "- However, the path can move above and left of C2, or below and right of C2."

                        "- For C3 (right bottom corner): The path must stay either on its right, on its bottom, or both on its right and bottom."
                        "- The path (waypoints of path) must **never** be directly in the region above and the left of C3."
                        "- However, the path can move below and right of C3, or above and left of C3."

                        "- For C4 (left bottom corner): The path must stay either on its left, on its bottom, or both on its left and bottom."
                        "- The path (waypoints of path) must **never** be directly in the region above and the right of C4."
                        "- However, the path can move below and left of C4, or above and right of C4."

                    "Make sure that in all cases, the path avoids entering the blue area by following the rules for each corner as described."
                    "Please ensure that no path points are placed on the edges of the blue area. All path points should be located entirely outside of blue area, avoiding any points along its boundaries."

                    
                   "3. **Generate a Path**: Create a trajectory according to the instructions. The path should include waypoints that account for all objects (starting at the yellow box, passing through the green box, and ending at the red box) while avoiding the blue area and make sure go underneath two below corneers and above two up corneers.\n\n"
                   
                   "Output Format:\n"
                   "Please generate the trajectory as a list of coordinates in the format [x, y], structured as a JSON object like this:\n"
                   "{\n"
                   "    \"Trajectory\": [\n"
                   "        [x1, y1],\n"
                   "        [x2, y2],\n"
                   "        ...\n"
                   "    ]\n"
                   "}\n\n"
                   
                   "Additional Notes:\n"
                   "- Ensure the JSON is properly formatted with correct syntax.\n"
                   "- Include curved motions where necessary to avoid the blue area.\n"
                   "- The path must consider the entire area of the blue square-shaped obstacle, avoiding it completely.\n"
                   "- Generare as many as points which is required to follow instruction in particular task 5"
    }
]

input_string = LLM(PROMPT_dis)
import textwrap
wrapped_string = textwrap.fill(input_string, width=100)  # Wrap at 50 characters per line
print(wrapped_string)